# PetroFit Demo: Petrosian Radii Using Abell 2744

In [ ]:
import warnings
import numpy as np
from copy import copy

from astropy.nddata import CCDData, Cutout2D
from astropy.stats import sigma_clipped_stats, sigma_clip
from astropy.utils.exceptions import AstropyWarning
from astropy.modeling import models

from photutils.segmentation import SourceCatalog 

In [ ]:
from petrofit.segmentation import make_catalog, plot_segments
from petrofit.photometry import source_photometry, order_cat
from petrofit.petrosian import Petrosian

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib import cm

plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams['image.origin'] = 'lower'

In [ ]:
import ipywidgets as widgets
from IPython.display import display

DISPLAY_STYLE = {'description_width': 'initial'}

# Load Data

In [ ]:
acs_filter_list = ['f435w', 'f606w', 'f814w']
wfc3_filter_list = ['f105w', 'f125w', 'f140w', 'f160w']

input_data_formatter = "https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0/hlsp_frontier_hst_{}-60mas_abell2744_{}_v1.0_drz.fits"

In [ ]:
data_paths = {}
for f in acs_filter_list:
    data_paths[f] = input_data_formatter.format('acs', f)
    
for f in wfc3_filter_list:
    data_paths[f] = input_data_formatter.format('wfc3', f)
    
petrosian_cat_collection = {}

In [ ]:
# Load data and vitals
# ---------------------

current_filter = 'f105w'
input_data_path = data_paths[current_filter]
#input_data_path = 'combined_data.fits'
data = CCDData.read(input_data_path, cache=True)

plt.imshow(data, vmin=0, vmax=data.data.std())

### Estimate data noise at dark area

In [ ]:
# Estimate data noise at dark area
# --------------------------------
noise_cutout = Cutout2D(data, (2760, 3420), 70)

noise_mean = noise_cutout.data.mean()
noise_sigma = noise_cutout.data.std()
noise_3_sigma = noise_sigma * 3.
noise_8_sigma = noise_sigma * 8.

print(noise_mean, noise_3_sigma, noise_8_sigma)

plt.imshow(noise_cutout.data, vmax=noise_mean+noise_3_sigma, vmin=noise_mean-noise_3_sigma)
plt.show()

n, bins, patches = plt.hist(noise_cutout.data.flatten(), bins=35, align='left', color='black')
plt.plot(bins[:-1], n, c='r', linewidth=3)

plt.xlabel('Flux Bins [{}]'.format(str(data.unit)))
plt.ylabel('Count')
plt.title('Noise Histogram')
plt.show()

# Target Image 

In [ ]:
# Cutout Image
# -------------

# Select and crop image:

cx, cy, size = 2083, 2859, 400 # Lone CD galaxy

image = Cutout2D(data, (cx, cy), size, copy=True)

# Compute image stats
image_min = image.data.min()
image_max = image.data.max()
image_mean = image.data.mean()
image_sigma = image.data.std()
image_3_sigma = image_sigma * 3.
image_8_sigma = image_sigma * 8.

image_clipped_mean, image_clipped_median, image_clipped_std = sigma_clipped_stats(image.data, sigma=3.0)

# Subtract Noise
#image.data  -= image_clipped_mean

# Set min and max values for all plots
vmin = -5 * image_clipped_std
vmax = +5 * image_clipped_std

plt.imshow(image.data, vmin=vmin, vmax=vmax)

print(image_mean)

# Segmentation (ID Target)

In [ ]:
# Define detect threshold
threshold = noise_8_sigma

# Define smoothing kernel
kernel_size = 5
fwhm = 5
npixels = 5**2


cat, segm, segm_deblend = make_catalog(
    image.data,
    threshold,
    deblend=True,
    kernel_size=kernel_size,
    fwhm=fwhm,
    npixels=npixels,
    plot=True, vmax=vmax, vmin=vmin
)

In [ ]:
result_cat = cat.to_table()
result_cat.sort(keys="area", reverse=1)

# Measure Photometry on Largest Object

We define the radii for Photometry

In [ ]:
# Define Radii
# ------------
# Define max rad in pixels and number of apertures
max_pix = 200
n = int(max_pix)//2

# Create list of radii
r_list = [x * int(max_pix / n) for x in range(1, n+1)]
r_list = np.array(r_list)

print("max_pix = {} pix".format(max_pix))
print("n = {}".format(n))
print("len(r_list) = {}".format(len(r_list)))

In [ ]:
# Sort and get the largest object in the catalog
idx = order_cat(cat)[0]  # index 0 is largest 
obj = cat[idx]  # get object (source) from the catalog 

# Photomerty 
flux_arr, area_arr, error_arr = source_photometry(
    obj, image.data, segm_deblend, r_list, 
    bkg_sub=True, sigma=1, sigma_type='clip',
    plot=True, vmax=vmax, vmin=vmin, 
    mask_background=False,
    cutout_size=max_pix, 
)
plt.show()

i.e this example does not set the cutout size automatically, if you would like to see a smaller object, decrease the `max_pix` var in the radii generation cell.

# Petrosian

### Construct Petrosian from Photometry

In [ ]:
p = Petrosian(r_list, area_arr, flux_arr)

### Petrosian radius

In [ ]:
p.r_petrosian # in pixels

### Petrosian total flux radius

In [ ]:
p.r_total_flux # pixels

In [ ]:
p.r_total_flux_arcsec(image.wcs) # arcsec

### Petrosian half light radius

In [ ]:
p.r_half_light # pixels

In [ ]:
p.r_half_light_arcsec(image.wcs) # arcsec

### Concentration Index

In [ ]:
r_20, r_80, c2080 = p.concentration_index()  # defualt c2080

r_20, r_80, c2080

In [ ]:
r_50, r_90, c5090 = p.concentration_index(fraction_1=0.5, fraction_2=0.9)  # defualt c2080

r_50, r_90, c5090

### Plot

In [ ]:
p.plot()

In [ ]:
# Plot with radii 
p.plot(plot_r=True)

In [ ]:
# Plot with radii and overplot normalized flux curve of growth
p.plot(plot_r=True, plot_normalized_flux=True)

### Change eta and epsilon

epsilon : float

    Epsilon value (used to determine `r_total_flux`).

    N.B: `r_total_flux` = `r_petrosian` * `epsilon`

eta : float, default=0.2

    Eta is the petrosian value which defines the `r_petrosian`.

In [ ]:
print('eta =', p.eta)
print('epsilon =', p.epsilon)
print('r_half_light =', p.r_half_light)
print('r_total_flux =', p.r_total_flux)

p.plot(plot_r=True)

In [ ]:
p_copy = copy(p)
p_copy.eta = 0.3
p_copy.epsilon = 3

print('eta =', p_copy.eta)
print('epsilon =', p_copy.epsilon)
print('r_half_light =', p_copy.r_half_light)
print('r_total_flux =', p_copy.r_total_flux)

p_copy.plot(plot_r=True)